In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from keras import (
    Input,
    Sequential,
    layers,
    models,
    callbacks,
    utils,
    metrics,
    optimizers,
)
from conlanger.models.WGANGP import WGANGP

In [ ]:
data = np.load("./data/language_phonemes.npz", allow_pickle=True)
language_phonemes_selected = data["language_phonemes_selected"]
language_names_selected = data["language_names_selected"]

# pad to 32x32
X = language_phonemes_selected.copy()
X = np.hstack([X, np.full((X.shape[0], 32 - X.shape[1], X.shape[2]), 0)])
X = np.dstack([X, np.full((X.shape[0], X.shape[1], 32 - X.shape[2]), 0)])
X = np.expand_dims(X, axis=3)

assert language_phonemes_selected.shape[0] == language_names_selected.shape[0]
(language_phonemes_selected.shape, language_names_selected.shape, X.shape)

In [ ]:
IMAGE_SIZE = 32
CHANNELS = 1
BATCH_SIZE = 128
NUM_FEATURES = 64
Z_DIM = 32 * 32
LEARNING_RATE = 0.0002
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.999
EPOCHS = 50
CRITIC_STEPS = 3
GP_WEIGHT = 10.0
LOAD_MODEL = False
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.9

In [ ]:
shape = X.shape[1:]
width = shape[0]

print(shape)

critic = Sequential(
    [
        Input(shape=shape),
        layers.Conv2D(IMAGE_SIZE, kernel_size=2, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(IMAGE_SIZE * 2, kernel_size=2, strides=2, padding="same"),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(IMAGE_SIZE * 4, kernel_size=2, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        layers.Conv2D(IMAGE_SIZE * 8, kernel_size=2, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        layers.Conv2D(IMAGE_SIZE * 16, kernel_size=2, strides=1, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        layers.Conv2D(1, kernel_size=2, strides=1, padding="valid"),
        layers.Flatten(),
    ],
    name="critic",
)
critic.summary()

In [ ]:
generator = Sequential(
    [
        Input(shape=(Z_DIM,)),
        layers.Reshape((1, 1, Z_DIM)),
        layers.Conv2DTranspose(
            256, kernel_size=4, strides=1, padding="valid", use_bias=False
        ),
        layers.Dropout(0.2),
        layers.Conv2D(
            256, kernel_size=3, strides=1, padding="same", use_bias=False
        ),
        layers.BatchNormalization(momentum=0.9),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(
            128, kernel_size=4, strides=2, padding="same", use_bias=False
        ),
        layers.Dropout(0.2),
        layers.Conv2D(
            128, kernel_size=3, strides=1, padding="same", use_bias=False
        ),
        layers.BatchNormalization(momentum=0.9),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(
            64, kernel_size=4, strides=2, padding="same", use_bias=False
        ),
        layers.Dropout(0.2),
        layers.Conv2D(
            64, kernel_size=3, strides=1, padding="same", use_bias=False
        ),
        layers.BatchNormalization(momentum=0.9),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(
            32, kernel_size=4, strides=2, padding="same", use_bias=False
        ),
        layers.Dropout(0.2),
        layers.Conv2D(32, kernel_size=3, strides=1, padding="same", use_bias=False),
        layers.BatchNormalization(momentum=0.9),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(
            32, kernel_size=4, strides=1, padding="same", use_bias=False
        ),
        layers.Dropout(0.2),
        layers.Conv2D(32, kernel_size=3, strides=1, padding="same", use_bias=False),
        layers.BatchNormalization(momentum=0.9),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(
            CHANNELS, kernel_size=4, strides=1, padding="same", activation="tanh"
        ),
    ],
    name="generator",
)
generator.summary()

In [ ]:
# Create a GAN
wgangp = WGANGP(
    critic=critic,
    generator=generator,
    latent_dim=Z_DIM,
    critic_steps=CRITIC_STEPS,
    gp_weight=GP_WEIGHT,
)

In [ ]:
# Compile the GAN
wgangp.compile(
    c_optimizer=optimizers.Adam(
        learning_rate=LEARNING_RATE, beta_1=ADAM_BETA_1, beta_2=ADAM_BETA_2
    ),
    g_optimizer=optimizers.Adam(
        learning_rate=LEARNING_RATE, beta_1=ADAM_BETA_1, beta_2=ADAM_BETA_2
    ),
)

In [ ]:
import matplotlib.pyplot as plt


class ImageGenerator(callbacks.Callback):
    def __init__(self, num_img, latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(
            shape=(self.num_img, self.latent_dim)
        )
        generated_images = self.model.generator(random_latent_vectors)
        generated_images = generated_images * 127.5 + 127.5
        generated_images = generated_images.numpy()
        plt.figure(figsize=(IMAGE_SIZE, IMAGE_SIZE))

        for i in range(10):
            _ = plt.subplot(1, 10, i + 1)
            plt.imshow(generated_images[i].astype("float32"), cmap="copper_r")
            plt.axis("off")
        
        plt.show()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(X)

wgangp.fit(
    dataset.repeat(),
    epochs=EPOCHS,
    steps_per_epoch=3,
    callbacks=[
        ImageGenerator(num_img=10, latent_dim=Z_DIM),
    ],
)